In [1]:
import oracledb, os, json
import pandas as pd
from google.cloud import secretmanager
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [3]:
def oracle_secrets():
  set_secrets_as_envs()
  return dict(
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    host = os.getenv('DBT_ORCL_HOST'),
    service = os.getenv('DBT_ORCL_SERVICE'),
    nb_url = os.getenv('NORGES_BANK_URL'),
    encoding="UTF-8",
    nencoding="UTF-8"
  )

oracle_secrets = oracle_secrets()

In [4]:
def categorize_financial_health(row):
    if row['Multiplikator'] == 'Hundre':
        return round(float(row['OBS_VALUE'].replace(',', '.'))/100, 2)
    else:
        return float(row['OBS_VALUE'].replace(',', '.'))

In [5]:
dagens_dato =  datetime.today().strftime("%Y-%m-%d")

url = oracle_secrets['nb_url'].format(dagens_dato) 

df = pd.read_csv(url, sep=';')

cols_to_show = [
    "TIME_PERIOD",
    "BASE_CUR",
    "QUOTE_CUR",
    "OBS_VALUE",
    "Multiplikator"
]

cols = [col for col in cols_to_show if col in df.columns]
df = df[cols]

df = df.rename(columns={'TIME_PERIOD': 'PERIODE', 'QUOTE_CUR': 'NOK_CUR'})

df['VALUTAKURSER'] = df.apply(categorize_financial_health, axis=1)
df['PERIODE'] = pd.to_datetime(df['PERIODE'], format='%Y-%m').dt.strftime('%Y%m')

In [6]:
cols_to_show = [
    "PERIODE",
    "BASE_CUR",
    "NOK_CUR",
    "VALUTAKURSER",
]

cols = [col for col in cols_to_show if col in df.columns]
df = df[cols]

In [9]:
user = oracle_secrets['user'] + '[DVH_FAM_BB]'
dsn_tns = oracledb.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])

with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
    with conn.cursor() as cursor:
        sql = """SELECT MAX(periode) FROM norges_bank_valuta"""
        cursor.execute(sql)
        result = cursor.fetchall()
        max_periode = result[0][0]
        
        df = df[df['PERIODE'] > str(max_periode)]
        
        rows = [tuple(x) for x in df.values]
        cursor.executemany('''INSERT INTO norges_bank_valuta(PERIODE,BASE_CUR,NOK_CUR,VALUTAKURSER)
                            VALUES (:1,:2,:3,:4)''', rows)
        conn.commit()